# Analyzing `nakeds`

## 1. Generate Nakeds
### Provide inputs

In [1]:
# * NAKED FUNCTION INPUTS
MARKET = 'NSE'
SYMBOL = 'RELIANCE'
EARLIEST = False
RECALC_UNDS = True
SAVE = False

RUN_ON_PAPER = True

In [2]:
# * IMPORTS
import IPython as ipy
import nest_asyncio
import pathlib

import pandas as pd
import numpy as np
from nakeds import get_nakeds
import datetime
from ib_insync import IB
from engine import Vars

import asyncio

# * SETUP

# . current working directory path
cwd = pathlib.Path.cwd()

# . get the host, port, cid
ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, CID = ibp.HOST, ibp.CID
if RUN_ON_PAPER:
    PORT = ibp.PAPER
else:
    PORT = ibp.PORT

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines

if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':

    from ib_insync import util
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    pd.options.display.float_format = '{:,.2f}'.format # set float precision with comma
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder
    BAR_FORMAT = "{l_bar}{bar:-20}{r_bar}"

In [4]:
# Get the pickle files
from os import listdir

cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files

fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_nakeds.pkl', 'df_ohlcs.pkl',
       'df_opt_prices.pkl', 'df_opts.pkl', 'df_symlots.pkl',
       'df_unds.pkl', 'dfrq.pkl', 'qopt_rejects.pkl', 'qopts.pkl',
       'z_temp_df_opt_prices.pkl'], dtype='<U24')

### Get the nakeds and ohlcs

In [8]:
drop_cols = ['contract', 'secType', 'comm', 
                    'lot', 'close', 'last', 'intrinsic']

df = df_nakeds.drop(drop_cols, 1)

m1 = df.bid>0
m2 = df.ask>0
df[m1&m2].head()
df.head()

,conId,symbol,dte,right,strike,expiry,und_iv,undPrice,und_sd,fallrise,fr_sd,margin,bid,ask,iv,price,sdMult,prop,timevalue,rom,expRom,expPrice,remq,qty
0,447334492,IDEA,16,P,1.00,20201231,0.99,10.05,4.32,2.00,3.85,"76,794.48",-1.00,-1.00,0.99,0.05,4.32,1.00,0.05,1.04,1.50,0.05,1,"70,000.00"
1,459328947,IBULHSGFI,44,C,280.00,20210128,0.86,194.45,1.48,646.10,7.82,"214,310.74",-1.00,-1.00,0.86,8.40,1.48,0.86,8.40,1.01,1.50,12.50,2,"3,100.00"
2,459328953,IBULHSGFI,44,C,285.00,20210128,0.86,194.45,1.57,646.10,7.82,"208,420.76",-1.00,-1.00,0.86,7.75,1.57,0.88,7.75,0.96,1.50,12.15,2,"3,100.00"
3,458881685,IBULHSGFI,44,C,275.00,20210128,0.86,194.45,1.39,646.10,7.82,"220,278.25",-1.00,-1.00,0.86,7.80,1.39,0.84,7.80,0.91,1.50,12.85,2,"3,100.00"
4,458355538,MOTHERSUM,44,C,202.50,20210128,0.59,156.50,1.45,255.25,3.10,"258,611.76",-1.00,-1.00,0.59,3.55,1.45,0.85,3.55,0.80,1.50,6.70,1,"7,000.00"


## 2. Plots
### a) OHLCs from pickle

### b) OHLC - live
#### The function

In [9]:
# .OHLC
async def ohlc(ib: IB,
               c, # contract
               DURATION: int = 365,
               OHLC_DELAY: int = 5) -> pd.DataFrame:

    "Gets OHLC history for stocks / options in UTC date" 

    if isinstance(c, tuple):
        c = c[0]

    if c.secType == 'OPT':
        DUR = "10 D"
        BAR_SIZE = "8 hours"
        WHAT_TO_SHOW = "MIDPOINT"

    else: # for non-options
        DUR = str(DURATION) + " D"
        BAR_SIZE = "1 day"
        WHAT_TO_SHOW = "TRADES"

    ohlc = await ib.reqHistoricalDataAsync(
        contract=c,
        endDateTime=datetime.datetime.now(),
        durationStr=DUR,
        barSizeSetting=BAR_SIZE,
        whatToShow=WHAT_TO_SHOW,
        useRTH=True,
        formatDate=2, # UTC format
    )
    await asyncio.sleep(OHLC_DELAY)
    df = util.df(ohlc)

    # re-cast OHLC for options
    if c.secType == 'OPT':

        # Introduce date without time
        df['dt'] = df['date'].dt.date
        
        # get the grouped values for the option
        g = df.groupby('dt')

        op = g.open.first()
        hi = g.high.max()
        lo = g.low.min()
        cl = g.close.last()
        vol = g.volume.sum()
        avg = g.average.mean()
        bc = g.barCount.sum()        

        df1 = pd.DataFrame({'date': df.dt.unique()})

        df = df1.assign(open=df1['date'].map(op),
                high=df1['date'].map(hi),
                low=df1['date'].map(lo),
                close=df1['date'].map(cl),
                volume=df1['date'].map(vol),
                average=df1['date'].map(avg),
                barCount=df1['date'].map(bc))

        df.insert(0, "localSymbol", c.localSymbol)
        df.insert(1, "strike", c.strike)
        df.insert(2, "right", c.right)
        df.insert(3, "expiry", c.lastTradeDateOrContractMonth)

    df.insert(0, "conId", c.conId)
    df.insert(1, "symbol", c.symbol)
    

    return df

#### Set up the contracts

In [18]:
# . underlying contracts
df_unds = pd.read_pickle(cwd.joinpath('data', MARKET.lower(), 'df_unds.pkl'))
und_ct = df_unds[df_unds.symbol == SYMBOL].contract.iloc[0]

# . option contracts

# ... get some with bid and ask
m1 = df_nakeds.bid>0
m2 = df_nakeds.ask>0
m3 = df_nakeds.symbol == SYMBOL

# if bid and ask price are not present (market closed), use last price
if df_nakeds[m1&m2&m3].empty:
    dfo = df_nakeds[m3]
else:
    dfo = df_nakeds[m1&m2&m3]


# ... choose the top one
opt_ct = dfo[dfo.conId == dfo.conId.iloc[0]].contract.iloc[0]

In [30]:
cols = "symbol	dte	right	strike	undPrice	expiry	contract	lot	und_iv	und_sd	fallrise	fr_sd	margin	bid	ask	close	last	iv	sdMult	prop	rom	expRom	price	expPrice	remq	qty".split('	')

In [31]:
dfo[cols]

,symbol,dte,right,strike,undPrice,expiry,contract,lot,und_iv,und_sd,fallrise,fr_sd,margin,bid,ask,close,last,iv,sdMult,prop,rom,expRom,price,expPrice,remq,qty
236,RELIANCE,44,C,"2,950.00","1,991.30",20210128,"Option(conId=452758118, symbol='RELIANCE', las...",250.00,0.38,3.70,"2,707.90",2.76,"48,426.60",-1.00,-1.00,1.90,nan,0.38,3.70,1.00,0.08,1.50,1.90,36.55,2,250.00
248,RELIANCE,44,C,"3,050.00","1,991.30",20210128,"Option(conId=452758135, symbol='RELIANCE', las...",250.00,0.38,4.08,"2,707.90",2.76,"44,692.23",-1.00,-1.00,1.45,nan,0.38,4.08,1.00,0.06,1.50,1.45,34.20,2,250.00
255,RELIANCE,44,C,"3,000.00","1,991.30",20210128,"Option(conId=452758124, symbol='RELIANCE', las...",250.00,0.38,3.89,"2,707.90",2.76,"46,460.98",-1.00,-1.00,1.45,nan,0.38,3.89,1.00,0.06,1.50,1.45,35.55,2,250.00
281,RELIANCE,44,C,"3,100.00","1,991.30",20210128,"Option(conId=454569142, symbol='RELIANCE', las...",250.00,0.38,4.27,"2,707.90",2.76,"43,104.73",-1.00,-1.00,1.10,nan,0.38,4.27,1.00,0.05,1.50,1.10,33.60,2,250.00
639,RELIANCE,16,P,850.00,"1,991.30",20201231,"Option(conId=453431055, symbol='RELIANCE', las...",505.00,0.38,7.30,"1,516.95",3.03,"66,057.24",-1.00,-1.00,0.05,nan,0.38,7.30,1.00,0.00,1.50,0.05,41.35,2,505.00
640,RELIANCE,16,P,900.00,"1,991.30",20201231,"Option(conId=453309437, symbol='RELIANCE', las...",505.00,0.38,6.98,"1,516.95",3.03,"66,101.43",-1.00,-1.00,0.05,nan,0.38,6.98,1.00,0.00,1.50,0.05,41.40,2,505.00
641,RELIANCE,16,P,950.00,"1,991.30",20201231,"Option(conId=453309445, symbol='RELIANCE', las...",505.00,0.38,6.66,"1,516.95",3.03,"66,297.11",-1.00,-1.00,0.05,nan,0.38,6.66,1.00,0.00,1.50,0.05,41.50,2,505.00
642,RELIANCE,16,P,"1,000.00","1,991.30",20201231,"Option(conId=452349418, symbol='RELIANCE', las...",505.00,0.38,6.34,"1,516.95",3.03,"66,934.68",-1.00,-1.00,0.05,nan,0.38,6.34,1.00,0.00,1.50,0.05,41.90,2,505.00
643,RELIANCE,16,C,"3,300.00","1,991.30",20201231,"Option(conId=449866949, symbol='RELIANCE', las...",505.00,0.38,8.37,"2,555.95",3.61,"67,875.24",-1.00,-1.00,0.05,nan,0.38,8.37,1.00,0.00,1.50,0.05,42.50,2,505.00
644,RELIANCE,16,C,"3,250.00","1,991.30",20201231,"Option(conId=447913148, symbol='RELIANCE', las...",505.00,0.38,8.05,"2,555.95",3.61,"68,620.11",-1.00,-1.00,0.05,nan,0.38,8.05,1.00,0.00,1.50,0.05,42.95,2,505.00


In [21]:
%%time
with IB().connect(HOST, PORT, CID )as ib:
    und_ohlcs = ib.run(ohlc(ib, und_ct, 365, 5))
    
with IB().connect(HOST, PORT, CID )as ib:
    opt_ohlcs = ib.run(ohlc(ib, opt_ct))

opt_ohlcs

Error 162, reqId 144: Historical Market Data Service error message:HMDS query returned no data: RELIANCE21JAN2950CE@NSE Midpoint, contract: Option(conId=452758118, symbol='RELIANCE', lastTradeDateOrContractMonth='20210128', strike=2950.0, right='C', multiplier='1', exchange='NSE', currency='INR', localSymbol='RELIANCE21JAN2950CE', tradingClass='RELIANCE')


TypeError: 'NoneType' object is not subscriptable

### Plot the und and option

In [ ]:
util.barplot(und_ohlcs, title=SYMBOL),

In [ ]:
util.barplot(opt_ohlcs, title=opt_ct.localSymbol),

In [ ]:
opt_ohlcs.head()

### option ohlcs
* Let us play around with option ohlcs

In [ ]:
async def opt_ohlc(ib: IB,
                   c, # contract
                   DURATION: str="2 W",
                   OHLC_DELAY: int = 5,
                   BAR_SIZE: str="8 hours",
                   WHAT_TO_SHOW: str="MIDPOINT") -> pd.DataFrame:

    "Gets option OHLC history in UTC date"
    if c.secType == 'OPT':
        DUR = DURATION
        BAR_SIZE = BAR_SIZE
        WHAT_TO_SHOW = WHAT_TO_SHOW

    else:
        print(f'\n{c} is not an option! It is {c.secType}')
        return None
    
    ohlc = await ib.reqHistoricalDataAsync(
        contract=c,
        endDateTime=datetime.datetime.now(),
        durationStr=DUR,
        barSizeSetting=BAR_SIZE,
        whatToShow=WHAT_TO_SHOW,
        useRTH=False,
        formatDate=2, # UTC format
    )
    await asyncio.sleep(OHLC_DELAY)
    df = util.df(ohlc)
    
    return df

In [ ]:
%%time
with IB().connect(HOST, PORT, CID )as ib:
    opt_ohlcs1 = ib.run(opt_ohlc(ib, opt_ct, DURATION='2 W', OHLC_DELAY=5,  BAR_SIZE="8 hours", WHAT_TO_SHOW="MIDPOINT"))

opt_ohlcs1

In [ ]:
util.barplot(opt_ohlcs1, title=opt_ct.localSymbol),